# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 3. Klasifikasi K-Nearest Neighbor (KNN)


Tuliskan Nama, NIM, dan kelas Anda:

Nama  :

NIM   :

Kelas :

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [2]:
import numpy as np
import pandas as pd
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-03-18 02:45:20--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-03-18 02:45:20 (53.6 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [3]:
iris_df = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [4]:
iris_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [5]:
# Pop label
label = iris_df.pop('variety')
# label

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [6]:
iris_df

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya **label_latih** dan **label_uji**

In [7]:
label

0         Setosa
1         Setosa
2         Setosa
3         Setosa
4         Setosa
         ...    
145    Virginica
146    Virginica
147    Virginica
148    Virginica
149    Virginica
Name: variety, Length: 150, dtype: object

In [9]:
from sklearn.model_selection import train_test_split
data_latih, data_uji, label_latih, label_uji = train_test_split(iris_df, label, test_size = 0.2, random_state = 101)
print(data_latih.shape)
print(data_uji.shape)

(120, 4)
(30, 4)


## 3) Menghitung jarak euclidean

Tahapan awal dari algoritma KNN adalah perhitungan jarak. Salah satu metode perhitungan jarak yang bisa digunakan adalah jarak Euclidean. Buatlah fungsi bernama **jarakEu** yang berfungsi menghitung jarak euclidean dari dua buah vektor (tanpa kelas)

In [10]:
def jarakEu(data1, data2):
  jarak = np.square(data1-data2) #kuadrat
  jarak = np.sum(jarak) #jumlahkan
  return np.sqrt(jarak)

Uji fungsi **jarakEu** untuk menghitung jarak antara data latih pertama dengan data uji pertama.

In [11]:
d1  = jarakEu(data_latih.iloc[0], data_uji.iloc[0])
print("Jarak data1 = ", d1)
data_latih.iloc[0]
data_uji.iloc[0]

Jarak data1 =  5.079370039680118


sepal.length    5.5
sepal.width     4.2
petal.length    1.4
petal.width     0.2
Name: 33, dtype: float64

## 4) Algoritma KNN

Implementasikan algoritma KNN dengan tahapan-tahapan berikut :
1. Hitung jarak antara data uji dengan setiap data latih.
2. Cari *k* data latih dengan jarak terkecil.
3. Tentukan kelas dari *k* data latih tersebut.
4. Kelas data uji ditentukan dari mayoritas kelas *k* data latih.

Buatlah fungsi dengan nama **knn** yang menerima input berupa *k*, sekumpulan data latih dan labelnya, serta sebuah data uji

In [12]:
from collections import Counter
# Hanya untuk 1 data uji
def knn(k,datalatih,labellatih,datauji):
  jarak = np.array([jarakEu(datalatih.iloc[x],datauji) for x in range(datalatih.shape[0])])
  indeks_k_minimum = jarak.argsort()[:k] #return the indek of array
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy() #search for the index
  counter = Counter(k_kelas)
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji


In [13]:
import numpy as np

arr = np.array([40, 10, 30, 20])
indeks_urut = np.argsort(arr)

print("Array Asli:", arr)
print("Indeks Urut:", indeks_urut)

Array Asli: [40 10 30 20]
Indeks Urut: [1 3 2 0]


In [14]:
jarak = np.array([1,4,8,0,2,5,3,7,6])
urutan = jarak.argsort()[:3]
urutan

array([3, 0, 4])

In [15]:
tes = ['a','a','a','b','c','a','b']
count_tes = Counter(tes)
print(count_tes)
count_tes.most_common(1)[0][0]

Counter({'a': 4, 'b': 2, 'c': 1})


'a'

Lakukan pengujian fungsi **knn** untuk menentukan kelas dari data uji pertama

In [16]:
predict = knn(3, data_latih, label_latih, data_uji.iloc[0])
predict
label_uji.iloc[0]

'Setosa'

## 5) KNN pada Scikit-learn

Library scikit-learn memiliki algoritma KNN yang siap untuk digunakan. Sebagai pengenalan, Anda akan melakukan klasifikasi data yang sama menggunakan KNN yang tersedia pada library scikit-learn

In [17]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(data_latih,label_latih)
kelas = KNN.predict([data_uji.iloc[0]])
print(kelas)

['Setosa']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


## TUGAS
1. Buatlah fungsi bernama **jarakCity** yang berfungsi menghitung jarak City Block antara dua vektor.
2. Buatlah sebuah fungsi bernama **knn_multi** yang dapat menentukan kelas dari **semua** data uji (Fungsi **knn** yang sudah Anda buat hanya dapat menentukan kelas dari sebuah data uji. Anda dapat memanfaatkan fungsi **knn** yang telah ada
3. Buatlah fungsi bernama **cek_hasil** yang berfungsi membandingkan hasil klasifikasi dengan label sebenarnya. Fungsi ini menerima input 2 buah list, yaitu list label hasil klasifikasi dan list label data uji yang sebenarnya. Output dari fungsi ini adalah integer, yang menunjukkan berapa banyak label hasil klasifikasi yang sama dengan label data uji sebenarnya.
4. Jalankan fungsi **knn_multi** dengan nilai k=3, k=7,k=9 dan k=13. Amati hasil dari fungsi **cek_hasil** dengan nilai k tersebut. Apa kesimpulan Anda?

In [27]:
def jarakCity(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("Panjang vektor harus sama")

    jarak = 0
    for i in range(len(v1)):
        jarak += abs(v1[i] - v2[i])

    return jarak


In [30]:
from collections import Counter

def knn_multi(data_latih, data_uji, k):

    def jarakCity(v1, v2):

        if len(v1) != len(v2):
            raise ValueError("Panjang vektor harus sama")

        jarak = 0
        for i in range(len(v1)):
            jarak += abs(v1[i] - v2[i])

        return jarak

    def knn_single(data_latih, data_uji, k):
        jarak_kelas = [(jarakCity(data_latih[i][0], data_uji), data_latih[i][1]) for i in range(len(data_latih))]
        jarak_kelas.sort(key=lambda x: x[0])
        k_kelas = [x[1] for x in jarak_kelas[:k]]
        counter = Counter(k_kelas)
        return counter.most_common(1)[0][0]

    hasil_knn = [knn_single(data_latih, data_uji[i][0], k) for i in range(len(data_uji))]
    return hasil_knn

In [31]:
def cek_hasil(hasil_klasifikasi, label_sebenarnya):

    count = 0
    for hasil, sebenarnya in zip(hasil_klasifikasi, label_sebenarnya):
        if hasil == sebenarnya:
            count += 1
    return count

In [32]:
label_sebenarnya = ['A', 'B']

list_k = [3, 7, 9, 13]

for k in list_k:
    hasil_knn = knn_multi(data_latih, data_uji, k)
    jumlah_benar = cek_hasil(hasil_knn, label_sebenarnya)
    print(f"Dengan k = {k}, jumlah label yang benar diprediksi:", jumlah_benar)

Dengan k = 3, jumlah label yang benar diprediksi: 1
Dengan k = 7, jumlah label yang benar diprediksi: 1
Dengan k = 9, jumlah label yang benar diprediksi: 1
Dengan k = 13, jumlah label yang benar diprediksi: 1


Penjelasan jawaban soal nomor 4

Program di atas melakukan eksperimen untuk mengamati kinerja model KNN dengan berbagai nilai k. Langkah-langkahnya adalah sebagai berikut:

Pertama-tama, data latih dan data uji telah ditentukan. Data latih adalah data yang digunakan untuk melatih model KNN, sedangkan data uji adalah data yang akan diprediksi kelasnya.
Label sebenarnya dari data uji juga telah ditentukan sebelumnya.
Kemudian, dilakukan iterasi melalui list nilai k yang akan diuji (list_k), yaitu [3, 7, 9, 13].
Pada setiap iterasi, dilakukan prediksi kelas untuk data uji menggunakan fungsi knn_multi dengan nilai k yang sesuai.
Hasil prediksi kemudian dibandingkan dengan label sebenarnya menggunakan fungsi cek_hasil untuk menghitung berapa banyak label yang benar diprediksi.
Jumlah label yang benar diprediksi untuk setiap nilai k kemudian dicetak.
Dengan membandingkan jumlah label yang benar diprediksi untuk berbagai nilai k, dapat ditarik kesimpulan tentang kinerja model KNN dengan nilai k yang berbeda. Semakin tinggi jumlah label yang benar diprediksi, semakin baik performa model KNN tersebut.